In [4]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import pandas as pd
from datetime import datetime
from google.oauth2.service_account import Credentials
import gspread
from google.cloud import storage
# client = storage.Client("MyFirstProject")

In [7]:
data = {
    "userid": [
    "10002536", "10008847", "10004679", "10023836",
    "10033752", "10038584", "10044023", "10192623",
    "10165926", "10093976", "10132431"
    ],
    "名前": [
        "キンシュウサイ", "大岡正樹", "池海龍（イケカイリュウ）", "飯島美桜",
        "江原ケイト", "樋口大輔", "加藤順平", "加藤奈津子",
        "松本明子", "石崎卓", "玉野利家"
    ],
    "PW": [
        "Kin20240301", "Hasegawa110", "Chihailong0803", "Cherry2525",
        "Tennis2784!", "46495963¥@Desu", "WCbifd@3NANwXGE", "aDA8a6MWs3XEz_Y",
        "Saikou1234", "Zaitaku2023", "Packen0731t"
    ]
}

user_df = pd.DataFrame(data)

COLUMNS = ["userid","予約番号", "利用日", "時間", "公園・施設", "設備予約", "支払状況"]

In [3]:
async def scrape_cancel_list(userid: str, pw: str) -> pd.DataFrame | None:
    p = await async_playwright().start()
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()

    try:
        # 1) Open + login
        await page.goto("https://kouen.sports.metro.tokyo.lg.jp/web/index.jsp", wait_until="domcontentloaded")

        await page.click("text=ログイン")
        await page.fill("#userId", userid)
        await page.fill("#password", pw)

        # Login often triggers navigation
        async with page.expect_navigation(wait_until="networkidle"):
            await page.click("#btn-go")

        # 2) Open reservation modal (as you already did)
        await page.click('a[data-target="#modal-reservation-menus"]')
        await page.wait_for_selector("#modal-reservation-menus", state="visible")

        # 3) Trigger the cancel/reservation list action ONCE and wait for navigation (if it happens)
        try:
            async with page.expect_navigation(wait_until="networkidle", timeout=10000):
                await page.evaluate("doAction(document.form1, gRsvWGetCancelRsvDataAction)")
        except PlaywrightTimeoutError:
            # In some flows it may update without a full navigation
            await page.wait_for_load_state("networkidle")

        # 4) If no reservations, tbody won't exist -> skip
        tbody = await page.query_selector("#rsvacceptlist tbody")
        if not tbody:
            rows = [[userid,"", "", "", "", "", ""]]
            return pd.DataFrame(rows, columns=COLUMNS)

        # 5) Extract first 6 columns (0..5) from each row
        rows = await page.evaluate("""
        [...document.querySelectorAll('#rsvacceptlist tbody tr')]
          .map(tr => [...tr.querySelectorAll('td')]
            .slice(0, 6)
            .map(td => td.innerText.trim()))
          .filter(r => r.length === 6)
        """)
        rows = [[userid, *r] for r in rows]

        return pd.DataFrame(rows, columns=COLUMNS)

    finally:
        await browser.close()
        await p.stop()


In [4]:
per_user_df_list = []
for index, row in user_df.iterrows():
    userid = str(row["userid"])
    pw = row["PW"]
    user_reservation_df = await scrape_cancel_list(userid, pw)
    per_user_df_list.append(user_reservation_df)
all_reservation_df = pd.concat(per_user_df_list, ignore_index=True)
today_ym = datetime.today().strftime("%Y/%m")
# all_reservation_df
user_df = user_df.merge(all_reservation_df, on="userid", how="left")
# user_df
user_df["extracted_date"] = today_ym


Exception: Browser.close: Connection closed while reading from the driver

In [8]:
# creds = Credentials.from_service_account_file(
#     "service_account.json",
#     scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
# )

# client = gspread.authorize(creds)

# sheet = client.open_by_url(
#     "https://docs.google.com/spreadsheets/d/14PQlmCB38Jc_YKABaQ3PU5d4KpTNHt6xIyOcLl7dscc"
# ).sheet1

# data = sheet.get_all_values()
managed_df = pd.read_excel("reservation_manage_sheet_python.xlsx", dtype=str)

df = pd.concat([managed_df, user_df], ignore_index=True)
df = df.apply(lambda col: col.str.strip())
df = df.apply(lambda c: c.astype(str).str.replace(r"[\r\n]", "", regex=True).str.strip())
df = df.astype(str)
df = df.drop_duplicates()


# # WRITE scopes (important)
# creds = Credentials.from_service_account_file(
#     "service_account.json",
#     scopes=[
#         "https://www.googleapis.com/auth/spreadsheets",
#         "https://www.googleapis.com/auth/drive"
#     ]
# )

# client = gspread.authorize(creds)

# sheet = client.open_by_url(
#     "https://docs.google.com/spreadsheets/d/14PQlmCB38Jc_YKABaQ3PU5d4KpTNHt6xIyOcLl7dscc"
# ).sheet1

# # overwrite content
# sheet.clear()
# sheet.update(
#     [df.columns.tolist()] + df.astype(str).values.tolist()
# )
